# Testing task with meat classification

## Dependencies

In [39]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=c27be7fa187e97f589d7ef051ebd5d02b2cd64becbbf4ef5720d1f31d58d0979
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [1]:
import time
import pandas as pd
import torch

from sklearn.model_selection import train_test_split

from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from torchtext.data.utils import get_tokenizer
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [59]:
MIN_LABELS = 500 # Min number of labels to form a class
EM_SIZE = 64 # Embedding size for text tokenization
TEST_SIZE = 0.2 # Train/test split
SEED = 42

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Custom dataset to load data into



In [67]:
class CustomTextDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file,sep=';')
        df.dropna(inplace = True)
        df = df[df.groupby('mtype')['mtype'].transform('count').ge(MIN_LABELS)]
        df.reset_index(inplace=True)
        
        labels = df.mtype.unique()
        self.labels_dict = {word:num for num, word in enumerate(labels)}
        self.targets = [self.labels_dict[label] for label in df.mtype]
        self.text = df.text        
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        label = self.targets[idx]
        text = self.text[idx]
        sample = (label, text)
        return sample

In [68]:
our_dataset = CustomTextDataset('https://axe.inline-ltd.ru/data/meatinfo.csv')

## Train/test split with equal proportions of classes

In [51]:
train_indices, test_indices, _, _ = train_test_split(
    range(len(our_dataset.targets)),
    our_dataset.targets,
    stratify=our_dataset.targets,
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subsets based on indices
train_split = Subset(our_dataset, train_indices)
test_split = Subset(our_dataset, test_indices)

In [52]:
tokenizer = get_tokenizer("moses",language='russian')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_split), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [55]:
text_pipeline = lambda x: vocab(tokenizer(x))

## Data processing pipeline

In [56]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(_label)
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

## Classification model

In [58]:
class MeatClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [60]:
num_class = len(set([label for (label, text) in train_split]))
vocab_size = len(vocab)
model = MeatClassificationModel(vocab_size, EM_SIZE, num_class).to(device)

## Training

In [83]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [94]:
# Hyperparameters
EPOCHS = 1 # epoch
LR = 1  # learning rate
BATCH_SIZE = 8 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataset = to_map_style_dataset(train_split)
test_dataset = to_map_style_dataset(test_split)
num_train = int(len(train_dataset))

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |    50/ 1562 batches | accuracy    0.993
| epoch   1 |   100/ 1562 batches | accuracy    0.995
| epoch   1 |   150/ 1562 batches | accuracy    0.995
| epoch   1 |   200/ 1562 batches | accuracy    0.988
| epoch   1 |   250/ 1562 batches | accuracy    0.990
| epoch   1 |   300/ 1562 batches | accuracy    0.993
| epoch   1 |   350/ 1562 batches | accuracy    0.990
| epoch   1 |   400/ 1562 batches | accuracy    0.978
| epoch   1 |   450/ 1562 batches | accuracy    0.990
| epoch   1 |   500/ 1562 batches | accuracy    0.988
| epoch   1 |   550/ 1562 batches | accuracy    0.993
| epoch   1 |   600/ 1562 batches | accuracy    0.990
| epoch   1 |   650/ 1562 batches | accuracy    0.993
| epoch   1 |   700/ 1562 batches | accuracy    0.995
| epoch   1 |   750/ 1562 batches | accuracy    0.988
| epoch   1 |   800/ 1562 batches | accuracy    0.993
| epoch   1 |   850/ 1562 batches | accuracy    0.988
| epoch   1 |   900/ 1562 batches | accuracy    0.995
| epoch   1 |   950/ 1562 ba

## Evaluation

In [95]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.947


In [74]:
meat_label = {y: x for x, y in our_dataset.labels_dict.items()}

In [70]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

In [71]:
model = model.to("cpu")

In [96]:
input_text = 'Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)\nСвободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.'
print("Это %s" %meat_label[predict(input_text, text_pipeline)])

Это Говядина


In [97]:
input_text = 'Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:'
print("Это %s" %meat_label[predict(input_text, text_pipeline)])

Это Кура


In [98]:
input_text = 'Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.'
print("Это %s" %meat_label[predict(input_text, text_pipeline)])

Это Говядина
